# 1. Setup Evaluation Date
---------------

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from QuantLib import *

plt.style.use("fivethirtyeight")

In [ ]:
Settings.instance().evaluationDate = Date(3, October, 2014)

# 2. Specifing Calibration Instruments
--------------

In [ ]:
helpers = [SwapRateHelper(QuoteHandle(SimpleQuote(rate/100.)),
                          Period(*tenor),
                          TARGET(),
                          Annual,
                          Unadjusted,
                          Thirty360(),
                          Euribor6M())
           for tenor, rate in [((2, Years), 0.201),
                               ((3, Years), 0.258),
                               ((5, Years), 0.464),
                               ((10, Years), 1.151),
                               ((15, Years), 1.588)]]

# 3. Buiding Yield Curve (from instruments)
-----------------------

In [ ]:
curve1 = PiecewiseFlatForward(0, TARGET(), helpers, Actual360())

In [ ]:
dates, rates = zip(*curve1.nodes())

In [ ]:
list(zip(dates, rates))

# 4. Building Yield Curve (from nodes)
------------------------------

In [ ]:
curve2 = ForwardCurve(dates, rates, Actual360())

In [ ]:
print(curve1.zeroRate(5.0, Continuous))
print(curve2.zeroRate(5.0, Continuous))

# 5. Plotting Curve
---------------

In [ ]:
times = np.linspace(0., 15., 400)
rates = [curve1.zeroRate(t, Continuous).rate() for t in times]

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(times, rates)

# 6. Moving the Evaluation Date
-----------------------

In [ ]:
Settings.instance().evaluationDate = Date(19, September, 2014)

In [ ]:
print(curve1.referenceDate())

In [ ]:
print(curve1.zeroRate(Date(7, September, 2019), Actual360(), Continuous))
print(curve2.zeroRate(Date(7, September, 2019), Actual360(), Continuous))

# 7. Notifications
-----------------

In [ ]:
def make_observer(i):
    def say():
        s = "Observer {0} notified".format(i)
        print('-' * len(s))
        print(s)
        print('-' * len(s))
    return Observer(say)

In [ ]:
obs1 = make_observer(1)
obs2 = make_observer(2)

In [ ]:
q1 = SimpleQuote(1.)
obs1.registerWith(q1)

q2 = SimpleQuote(2.)
obs2.registerWith(q2)

q3 = SimpleQuote(3.)
obs1.registerWith(q3)
obs2.registerWith(q3)

In [ ]:
q1.setValue(1.5)

In [ ]:
q2.setValue(1.9)

In [ ]:
q3.setValue(3.1)

In [ ]:
obs1.registerWith(curve1)
obs2.registerWith(curve2)

In [ ]:
Settings.instance().evaluationDate = Date(23, September, 2014)